## Code for country_series.csv

In [0]:
!pip install tika

In [0]:
from google.colab import drive, files
import os
import shutil
import pandas as pd
import time
from bs4 import BeautifulSoup
import urllib.request
from requests import get
from tika import parser
import numpy as np

In [0]:
drive.mount('/content/drive')

In [0]:
uploaded = files.upload()
!ls

### Scrape CPI 1995 - 2018 values 

In [0]:
#CPIs stored in different formats for different years
#split into 4 partitions
#1995 - 1997
#1998 - 2009
#2010 - 2011
#2012 - 2018
#values are positioned differently for different pdfs or html so tweak the array indexing to obtain correct CPI values

def get_country_cpi(country): #country (String type)
  url_1996 = 'https://www.transparency.org/files/content/tool/1996_CPI_EN.pdf'
  url_1997 = 'https://www.transparency.org/files/content/tool/1997_CPI_EN.pdf'
  url_2010 = 'http://files.transparency.org/content/download/132/531/2010_CPI_EN.pdf'
  url_2011 = 'http://files.transparency.org/content/download/101/407/file/2011_CPI_EN.pdf'
  
  diff = 2009 - 1998 + 1
  years = [i+1998 for i in range(diff)]

  pdf_urls = [url_1996,url_1997,url_2010,url_2011]
  vals_1995_1997 = []
  lines = []

  #1996 - 1997
  for i in pdf_urls[:2]:
    for j in parser.from_file(i)['content'].strip('\n').split('\n'):
      if country in j:
        print(j)
        lines += [j]
      
  if not lines:
    return f'cant find {country} 1996'
  print("")

  tmp = lines[0].split()
  vals_1995_1997 += [tmp[3]]
  vals_1995_1997 += [tmp[2]]

  vals_1995_1997 += [lines[7].split()[2]]

  for i in range(len(vals_1995_1997)):
    vals_1995_1997[i] = round(float(vals_1995_1997[i].replace(',','.')) *10,1)
  print('for 1995-1997')
  print(vals_1995_1997)


  lines = []
  vals_2010_2011 = []

  for i in pdf_urls[2:]:
    for j in parser.from_file(i)['content'].strip('\n').split('\n'):
      if country in j:
        print(j)
        lines += [j]

  if not lines:
    return f'cant find {country} 2010'
  #2010 - 2011

  #australia
  vals_2010_2011 += [lines[0].split()[2]]
  vals_2010_2011 += [lines[2].split()[2]]


  for i in range(len(vals_2010_2011)):
    vals_2010_2011[i] = round(float(vals_2010_2011[i]) * 10,1)

  print('for 2010-2011')
  print(vals_2010_2011)


  #1998 - 2009
  vals_1998_2009 = []
  for i in years:
    response = get(f'https://www.transparency.org/research/cpi/cpi_{i}/0')
    soup = BeautifulSoup(response.text, 'html.parser')
    for j in soup.find_all('tr'):
      if country in j.text:
        print(j.text.split('\n'))
        #2002 - 2
        #if 2002 <= i <= 2005:
        if  i == 2005:
          vals_1998_2009 += [j.text.split('\n')[2]]
        #elif i == 2001 or i == 1999:
        #elif i == 2001 or i==2003 or i==2005:
          #vals_1998_2009 += [j.text.split('\n')[5]]
        else:
          vals_1998_2009 += [j.text.split('\n')[3]]

  if  not vals_1998_2009:
    return f'cannot find {country} for 1998'

  print('before 1998-2009') 
  print(vals_1998_2009)

  for i in range(len(vals_1998_2009)):
    vals_1998_2009[i] = round(float(vals_1998_2009[i]) * 10,1)

  print('after 1998-2009')
  print(vals_1998_2009)
  #2012 - 2018
  df = pd.read_csv('CPI_2012-2018.csv', header=0)
  df = df.fillna(method='ffill')

  index = list(df['Country']).index(country)
  vals_2012_2018 = list(df.iloc[index][1:])
  
  vals = vals_1995_1997 + vals_1998_2009 + vals_2010_2011 + vals_2012_2018
  return vals


### Build time series for a specific country

In [0]:
out = pd.DataFrame()
start  = time.time()
with os.scandir(f'drive/My Drive/CDS_project/data/NGA') as dir:
  for indicator in dir:
    #print(indicator.name)
    df = pd.read_csv(indicator, header=0, index_col=0)
    if not df.empty:
      #print('entered')
      if out.empty:
        #print('out is empty')
        out = df
      else:
        out = out.join(df)

end = time.time() - start
print(end)
out

266.08454632759094


,NGA_TM_TAX_MANF_BR_ZS,NGA_TM_TAX_MRCH_BR_ZS,NGA_FM_LBL_BMNY_ZG,NGA_FM_LBL_BMNY_GD_ZS,NGA_FM_LBL_BMNY_CN,NGA_FM_LBL_BMNY_IR_ZS,NGA_GC_BAL_CASH_GD_ZS,NGA_IC_BUS_DISC_XQ,NGA_IQ_WEF_CUST_XQ,NGA_GC_BAL_CASH_CN,NGA_GC_DOD_TOTL_GD_ZS,NGA_AG_YLD_CREL_KG,NGA_AG_PRD_CREL_MT,NGA_GC_DOD_TOTL_CN,NGA_NE_GDI_STKB_CN,NGA_NE_GDI_STKB_KN,NGA_BN_RES_INCL_CD,NGA_SL_AGR_0714_ZS,NGA_NE_GDI_STKB_CD,NGA_NV_MNF_CHEM_ZS_UN,NGA_SL_AGR_0714_MA_ZS,NGA_SL_MNF_0714_ZS,NGA_SL_AGR_0714_FE_ZS,NGA_SL_MNF_0714_FE_ZS,NGA_SL_SRV_0714_ZS,NGA_SL_SRV_0714_FE_ZS,NGA_SL_MNF_0714_MA_ZS,NGA_SL_SRV_0714_MA_ZS,NGA_SE_PRM_UNER_FE,NGA_SE_PRM_UNER,NGA_SH_HIV_0014,NGA_SE_PRM_UNER_MA,NGA_FM_AST_CGOV_ZG_M3,NGA_SH_MLR_TRET_ZS,NGA_FS_AST_DOMO_GD_ZS,NGA_FS_AST_CGOV_GD_ZS,NGA_FM_AST_PRVT_ZG_M3,NGA_FM_AST_DOMO_ZG_M3,NGA_EN_ATM_CO2E_KD_GD,NGA_EN_ATM_CO2E_PP_GD_KD,...,NGA_NV_AGR_TOTL_KD,NGA_NV_AGR_TOTL_KD_ZG,NGA_NV_AGR_TOTL_ZS,NGA_NV_AGR_TOTL_CN,NGA_NV_AGR_TOTL_KN,NGA_IS_AIR_PSGR,NGA_NV_AGR_TOTL_CD,NGA_IS_AIR_GOOD_MT_K1,NGA_ER_H2O_FWAG_ZS,NGA_EG_USE_COMM_CL_ZS,NGA_IS_AIR_DPRT,NGA_ER_H2O_FWIN_ZS,NGA_ER_H2O_FWDM_ZS,NGA_ER_H2O_FWTL_ZS,NGA_ER_H2O_FWTL_K3,NGA_AG_LND_ARBL_ZS,NGA_SH_HIV_ARTC_ZS,NGA_AG_LND_ARBL_HA,NGA_AG_LND_ARBL_HA_PC,NGA_SH_STA_ARIC_ZS,NGA_MS_MIL_TOTL_TF_ZS,NGA_MS_MIL_TOTL_P1,NGA_FB_ATM_TOTL_P5,NGA_MS_MIL_MPRT_KD,NGA_MS_MIL_XPRT_KD,NGA_AG_LND_PRCP_MM,NGA_IC_CUS_DURS_EX,NGA_IC_TAX_METG,NGA_FB_BNK_CAPA_ZS,NGA_FD_RES_LIQU_AS_ZS,NGA_FB_AST_NPER_ZS,NGA_VC_BTL_DETH,NGA_TM_TAX_MRCH_BC_ZS,NGA_TM_TAX_MANF_BC_ZS,NGA_TM_TAX_TCOM_BC_ZS,NGA_SP_DYN_CBRT_IN,NGA_EN_BIR_THRD_NO,NGA_FB_CBK_BRWR_P3,NGA_SH_STA_BRTC_ZS,NGA_TM_TAX_TCOM_BR_ZS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1995-12-31,68.44,115.29,19.411715,10.281913,3.187635e+11,2.650768,NaN,NaN,NaN,NaN,NaN,1210.5,22512600.0,NaN,1.045843e+09,8.940942e+09,-8.626524e+08,NaN,1.486417e+07,25.465605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63000.0,NaN,-9.406721,NaN,NaN,8.483317,22.335131,NaN,0.246183,0.110976,...,2.646331e+10,3.586410,25.486506,7.901416e+11,3.977382e+12,5.479000e+05,1.122998e+10,1.900000,NaN,0.381946,6600.000000,NaN,NaN,NaN,NaN,36.123280,NaN,32900000.0,0.304597,NaN,0.266649,89100.0,NaN,2000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.53,11.25,56.58,43.443,NaN,NaN,NaN,146.76
1996-12-31,67.44,115.01,16.178159,9.063329,3.703335e+11,1.069242,NaN,NaN,NaN,NaN,NaN,1220.4,21665000.0,NaN,1.793278e+09,1.227920e+10,-2.384873e+08,NaN,2.241597e+07,25.706182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75000.0,NaN,-47.852793,NaN,NaN,2.703470,15.844491,NaN,0.261482,0.117873,...,2.750235e+10,3.926359,26.199159,1.070515e+12,4.133548e+12,2.212000e+05,1.338143e+10,4.700000,NaN,0.343117,5600.000000,NaN,NaN,NaN,NaN,36.342875,NaN,33100000.0,0.298917,NaN,0.260236,89100.0,NaN,15000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,21.48,11.15,57.28,43.363,NaN,NaN,NaN,146.82
1997-12-31,67.44,115.01,16.039003,9.725269,4.297313e+11,0.680632,NaN,NaN,NaN,NaN,NaN,1182.8,21853000.0,NaN,2.439006e+09,1.540889e+10,4.670708e+06,NaN,3.005924e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88000.0,NaN,-17.310651,NaN,NaN,1.049139,15.761332,NaN,0.271070,0.122195,...,2.864762e+10,4.164253,27.416651,1.211462e+12,4.305680e+12,3.182000e+05,1.493051e+10,5.000000,NaN,0.332273,6400.000000,NaN,NaN,NaN,NaN,36.452672,NaN,33200000.0,0.292453,NaN,0.304652,107000.0,NaN,1000000.0,NaN,1150.0,NaN,NaN,NaN,NaN,NaN,NaN,21.48,11.15,57.28,43.303,NaN,NaN,NaN,146.82
1998-12-31,67.44,115.01,22.317758,10.939035,5.256377e+11,0.818404,NaN,NaN,NaN,NaN,NaN,1201.8,22040000.0,NaN,2.957541e+09,1.819606e+10,-8.184873e+08,NaN,3.360842e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100000.0,NaN,21.771239,NaN,NaN,2.911794,12.906972,NaN,0.236412,0.106572,...,2.977579e+10,3.938095,27.908371,1.341041e+12,4.475241e+12,3.134000e+05,1.523910e+10,27.000000,NaN,0.350200,7500.000000,NaN,NaN,NaN,NaN,37.331050,NaN,34000000.0,0.292132,NaN,0.296818,107000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.400000,NaN,21.48,11.15,57.28,43.254,NaN,NaN,NaN,146.82
1999-12-31,67.44,115.01,33.121058,12.76339

In [0]:
a = get_country_cpi('Nigeria')

2019-12-03 16:14:19,942 [MainThread  ] [INFO ]  Retrieving https://www.transparency.org/files/content/tool/1996_CPI_EN.pdf to /tmp/files-content-tool-1996_cpi_en.pdf.
2019-12-03 16:14:21,066 [MainThread  ] [INFO ]  Retrieving https://www.transparency.org/files/content/tool/1997_CPI_EN.pdf to /tmp/files-content-tool-1997_cpi_en.pdf.


54 Nigeria 0,69  6,37 4 
Is (e.g.) Nigeria the most corrupt country in the world?  
No! Nigeria is perceived by business people to be the most corrupt country which has been 
than Nigeria. Also the perception of corruption must not necessarily reflect the real level of 
corruption a justifiable business in countries like Nigeria. Also in Nigeria corruption is sensed to 
54 Nigeria 0,69 4 0,63 2 0,99 2 


2019-12-03 16:14:21,832 [MainThread  ] [INFO ]  Retrieving http://files.transparency.org/content/download/132/531/2010_CPI_EN.pdf to /tmp/content-download-132-531-2010_cpi_en.pdf.


For the second year running, Nigeria has emerged in the lowest position and is thus perceived 
52 Nigeria 1,76 0,69 4 0,16 
Is (e.g.) Nigeria the most corrupt country in the world?
No! Nigeria is perceived by contributors to be the most corrupt in this ranking of 52
worse than Nigeria. Also the perception of corruption must not necessarily reflect the
points. This means that there is even stronger agreement that Nigeria scores lowest
included due to a lack of data. Nigeria is still bottom, and although its very low score

for 1995-1997
[63.7, 6.9, 17.6]


2019-12-03 16:14:23,776 [MainThread  ] [INFO ]  Retrieving http://files.transparency.org/content/download/101/407/file/2011_CPI_EN.pdf to /tmp/content-download-101-407-file-2011_cpi_en.pdf.


134 Nigeria 2.4
134 28 Nigeria 2.4 2.2 2.7 7
143 Nigeria 2.4
for 2010-2011
[24.0, 24.0]
['', '81', 'Nigeria', '1.9', '0.5', '5', '']
['', '98', 'Nigeria', '1.6', '0.8', '5', '']
['', '90', 'Nigeria', '1.2', '4', '0.6', '0.6 - 2.1', '']
['', '90', 'Nigeria', '1.0', '4', '0.9', '-0.1 - 2.0', '']
['', '101', 'Nigeria', '1.6', '6', '0.6', '', '0.9 - 2.5', '', '']
['', '132', 'Nigeria', '1.4', '9', '0.4', '0.9 - 2.0', '']
['', '144', 'Nigeria', '1.6', '1.4 - 1.8', '9', '']
['', 'Nigeria', '1.9', '1.7 - 2.0', '9', '']
['', '142', 'Nigeria', '2.2', '7', '2.0 - 2.3', '']
['', '147', 'Nigeria', '2.2', '8', '2.0 - 2.4', '']
['', '121', 'Nigeria', '2.7', '7', '2.3 - 3.0', '']
['', '130', 'Nigeria', '2.5', '7', '2.2 - 2.7', '']
before 1998-2009
['1.9', '1.6', '1.2', '1.0', '1.6', '1.4', '1.6', '1.9', '2.2', '2.2', '2.7', '2.5']
after 1998-2009
[19.0, 16.0, 12.0, 10.0, 16.0, 14.0, 16.0, 19.0, 22.0, 22.0, 27.0, 25.0]


### Combine CPI and indicators and output the time series as csv

In [0]:
out.index = pd.to_datetime(out.index)
out.index.year
common = np.intersect1d( years,out.index.year, assume_unique=True)
year_index = {years[i] : i for i in range(len(years))}
index_arr = [year_index[i] for i in common]
this_cpi = [a[i] for i in index_arr]
print(common)
this_cpi

In [0]:
out_new.to_csv('nga_series.csv')
shutil.copy('nga_series.csv', 'drive/My Drive/CDS_project/')